In [22]:
import numpy as np
import pandas as pd
import getpass
import os
import sys

from datetime import datetime
from time import sleep
from random import SystemRandom

try:
    import robin_stocks.robinhood as r
    import robin_stocks.robinhood.helper as rh
except ImportError:
    pass

In [2]:
RAND = SystemRandom()
ACTIONS = {1: [0.6, 0.4],
           2: [0.57, 0.43],
           3: [0.65, 0.35],
           4: [0.75, 0.25],
           5: [0.48, 0.52],
           6: [0.6, 0.4],
           8: [0, 1],
           9: [0.7, 0.3],
           10: [0.53, 0.47],
           13: [0.76, 0.24],
           14: [0.76, 0.24],
           15: [0.65, 0.35],
           17: [0.64, 0.36],
           18: [0.59, 0.41],
           19: [1, 0],
           22: [0.14, 0.86],
           23: [0.67, 0.33],
           25: [0, 1],
           26: [0.7, 0.3],
           27: [0.71, 0.29],
           28: [0.27, 0.44],
           29: [0.23, 0.4],
           30: [0.33, 0.33],
           32: [0.4, 0.4],
           33: [0.26, 0.53],
           36: [0.6, 0.3],
           37: [0.19, 0.55],
           38: [0, 0.6],
           40: [0.3, 0.41],
           41: [0.27, 0.5],
           42: [0.21, 0.46],
           43: [1, 0],
           44: [0.29, 0.57],
           45: [0.29, 0.47],
           46: [0, 0.67],
           49: [0.19, 0.38],
           50: [0.15, 0.62],
           52: [0, 0.5],
           53: [0.42, 0.46],
           54: [0.25, 0.44],
           55: [0, 0.3],
           56: [0, 0.33],
           57: [0, 0],
           59: [0, 0],
           60: [0, 0.25],
           63: [0, 1],
           64: [0, 0.4],
           65: [0, 0.33],
           67: [0, 0.38],
           68: [0, 0.35],
           69: [0, 0.35],
           72: [0, 0.44],
           73: [0, 0.32],
           74: [0, 0.5],
           76: [0, 0.36],
           77: [0, 0.38],
           78: [0, 0.64],
           79: [0, 0.48],
           80: [0, 0.36],
           81: [0, 0.42]}

In [3]:
username = 'watsona4@gmail.com'
password = getpass.getpass()

········


In [4]:
login = r.login(username, password)

In [5]:
h = r.get_crypto_historicals('BTC', interval='5minute', span='day')

In [6]:
df = pd.DataFrame(h)
df['begins_at'] = pd.to_datetime(df['begins_at']).dt.tz_localize(None)

In [7]:
df.to_csv(f'btc_{datetime.now():%Y-%m-%d_%H%M%S}.csv')

In [8]:
def get_holdings():
    positions = r.get_crypto_positions()
    for pos in positions:
        if pos['currency']['code'] == 'BTC':
            return float(pos['quantity'])

In [9]:
def get_value():
    profile = r.load_account_profile()
    return float(profile['portfolio_cash'])

In [26]:
etime = lambda x: (x - datetime(1900, 1, 1)).total_seconds()/3600/24

def get_next_price():
    sleep(300)
    quote = r.get_crypto_quote('BTC')
    print(f'{datetime.now()}: mark={quote["mark_price"]}, ask={quote["ask_price"]}, bid={quote["bid_price"]}')
    return {'time': etime(datetime.now()),
            'mark': float(quote['mark_price']),
            'ask': float(quote['ask_price']),
            'bid': float(quote['bid_price'])}

In [24]:
def get_action(box):
    if box not in ACTIONS:
        return 'HOLD'
    b = ACTIONS[box]
    rand = RAND.random()
    if rand < b[0]:
        return 'BUY'
    if rand < b[0] + b[1]:
        return 'HOLD'
    return 'SELL'

In [ ]:
p0 = pp1 = pp2 = pp3 = None

while True:

    p1 = p0
    p0 = get_next_price()

    if p1 is None: continue

    dt = p0['time'] - p1['time']

    pp1old = pp1
    pp1 = (p0['mark'] - p1['mark'])/(p0['mark']*dt)

    if pp1old is None: continue

    pp2old = pp2
    pp2 = (pp1 - pp1old)/(500*dt)

    if pp2old is None: continue

    pp3old = pp3
    pp3 = (pp2 - pp2old)/(500*dt)

    if pp3old is None: continue

    pp4 = (pp3 - pp3old)/(500*dt)

    print(f'{datetime.now()}: pp1 = {pp1:.4f}, pp2 = {pp2:.4f}, pp3 = {pp3:.4f}, pp4 = {pp4:.4f}')

    def boxpp(x):
        if x < -0.2:
            return 0
        if x < 0.2:
            return 1
        return 2

    box = 27*boxpp(pp1) + 9*boxpp(pp2) + 3*boxpp(pp3) + boxpp(pp4) + 1

    action = get_action(box)

    print(f'{datetime.now()}: action={action:4s}, shares={get_holdings()}, value={get_value()}')
    
    if action == 'BUY':
        r.order_buy_crypto_by_price('BTC', get_value())
            if 'account_id' not in order:
                print(order)
    elif action == 'SELL':
        if get_holdings() > 1e-6:
            order = r.order_sell_crypto_by_quantity('BTC', get_holdings())
            if 'account_id' not in order:
                print(order)

2021-04-23 05:02:15.074804: mark=48604.640000, ask=48624.090000, bid=48585.190000
2021-04-23 05:07:16.536766: mark=48518.305000, ask=48537.720000, bid=48498.890000
2021-04-23 05:12:16.897795: mark=48632.164800, ask=48659.972606, bid=48604.356995
2021-04-23 05:17:17.249569: mark=48392.340000, ask=48411.440000, bid=48373.240000
2021-04-23 05:22:17.605975: mark=48260.922406, ask=48273.160000, bid=48248.684813
2021-04-23 05:22:17.605975: pp1 = -0.7833, pp2 = 0.3695, pp3 = 0.9074, pp4 = 1.1471
2021-04-23 05:22:17.605975: action=BUY , shares=0.00169, value=0.19
2021-04-23 05:27:18.306651: mark=48294.663658, ask=48310.107317, bid=48279.220000
2021-04-23 05:27:18.306651: pp1 = 0.2007, pp2 = 0.5655, pp3 = 0.1126, pp4 = -0.4567
2021-04-23 05:27:18.306651: action=HOLD, shares=0.00169, value=0.19
2021-04-23 05:32:18.833292: mark=48480.640248, ask=48496.790000, bid=48464.490497
2021-04-23 05:32:18.833292: pp1 = 1.1029, pp2 = 0.5187, pp3 = -0.0269, pp4 = -0.0802
2021-04-23 05:32:18.833292: action=SE